In [2]:
#Import Required Libraries
import pandas as pd
import nltk

In [4]:
df_imdb = pd.read_csv('C:\DS\Datasets\imdb_labelled\imdb_labelled.txt', sep='\t', names = ['comments','label'])

In [5]:
df_imdb.head(10)

,comments,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


In [6]:
df_imdb.describe()

,label
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
comments    748 non-null object
label       748 non-null int64
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [8]:
# View some extra information about the dataset
df_imdb.groupby('label').describe()

comments                                
         count unique                 top freq
label                                         
0          362    361  Not recommended.      2
1          386    384             10/10      2

In [9]:
df_imdb['length'] = df_imdb['comments'].apply(len)
df_imdb.head()

,comments,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [12]:
# View the first message whose length > 100 (Just experimental; No significance)
df_imdb[df_imdb['length']>100]['comments'].iloc[0]

'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  '

In [13]:
# Importing CountVextorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()


In [19]:
# Defining a Function to get rid of Punctuations and Stopwords
import string
def message_text_process(mess):
    # Eliminating Punctuations
    no_punc = [char for char in mess if char not in string.punctuation]
    # Forming a Sentence
    no_punc = ''.join(no_punc)
    # Eliminating Stopwords
    return [word for word in no_punc.split() if word not in stopwords.words('english')]

In [20]:
# Applying Bag of Words, fitting and transforming the data
from nltk.corpus import stopwords
bag_of_words = CountVectorizer(analyzer=message_text_process).fit(df_imdb['comments'])

In [21]:
comment_bagofwords = bag_of_words.transform(df_imdb['comments'])

In [22]:
# Apply the tf-idf transformer
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(comment_bagofwords)

In [26]:
# Printing the Shape of the Tf-idf Transformer
comment_tdif = tfidf_transformer.transform(comment_bagofwords)
print(comment_tdif)

<748x3340 sparse matrix of type '<class 'numpy.float64'>'
	with 7743 stored elements in Compressed Sparse Row format>

In [27]:
# Create a Namive Bayes Model to detect sentiment
from sklearn.naive_bayes import MultinomialNB
imdb_model = MultinomialNB().fit(comment_tdif, df_imdb['label'])

In [28]:
# Evaluate the Model's Accuracy
comment = df_imdb['comments'][0]

In [31]:
# Final Check based on Predicted and Actual Values
comment = df_imdb['comments'][0]
bag_of_words_for_comment = bag_of_words.transform([comment])
tfidf = tfidf_transformer.transform(bag_of_words_for_comment)

print('Comment: ', comment)
print('Predicted Sentiment:', imdb_model.predict(tfidf)[0])
print('Expected Sentiment:', df_imdb.label[0])

Comment:  A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  
Predicted Sentiment: 0
Expected Sentiment: 0
